In [1]:
from abc import ABC, abstractmethod
from typing import Dict, List
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from tqdm.auto import tqdm
import ast
from catboost import CatBoostRegressor, Pool

In [2]:
data_folder = "./dataset_additional/"

users_df = pd.read_csv(data_folder + "users_df.csv")
items_df = pd.read_csv(data_folder + "items_df.csv")

countries = pd.read_csv(data_folder + "countries.csv")
genres = pd.read_csv(data_folder + "genres.csv")
staff = pd.read_csv(data_folder + "staff.csv")

data_folder = "./"

train_part = pd.read_csv(data_folder + "train_data.csv", parse_dates=["datetime"])
test_part = pd.read_csv(data_folder + "test_data.csv")
test_part = test_part.groupby("user_id").agg({"movie_id": list}).reset_index()

In [3]:
class BaseRecommender(ABC):
    def __init__(self):
        self.trained = False

    @abstractmethod
    def fit(self, df: pd.DataFrame) -> None:
        # реализация может быть любой, никаких ограничений

        # не забудьте про
        self.trained = True

    @abstractmethod
    def predict(self, df: pd.DataFrame, topn: int = 10) -> List[np.ndarray]:
        # реализация может быть любой, НО
        # должен возвращать список массивов из movie_id, которые есть в `item_df`, чтобы корректно работал подсчет метрик
        pass

In [4]:
# ACHTUNG! DO NOT TOUCH 

def ndcg_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    at = len(predicted)
    relevance = np.array([1 if x in predicted else 0 for x in gt_items])
    # DCG uses the relevance of the recommended items
    rank_dcg = dcg(relevance)
    if rank_dcg == 0.0:
        return 0.0

    # IDCG has all relevances to 1 (or the values provided), up to the number of items in the test set that can fit in the list length
    ideal_dcg = dcg(np.sort(relevance)[::-1][:at])

    if ideal_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg

    return ndcg_


def dcg(scores: np.ndarray) -> float:
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)), dtype=np.float64
    )


def recall_metric(gt_items: np.ndarray, predicted: np.ndarray) -> float:
    n_gt = len(gt_items)
    intersection = len(set(gt_items).intersection(set(predicted)))
    return intersection / n_gt


def evaluate_recommender(df: pd.DataFrame, model_preds_col: str, gt_col: str = "movie_id") -> Dict[str, float]:
    metric_values = []

    for _, row in df.iterrows():
        metric_values.append(
            (ndcg_metric(row[gt_col], row[model_preds_col]), recall_metric(row[gt_col], row[model_preds_col]))
        )

    return {"ndcg": np.mean([x[0] for x in metric_values]), "recall": np.mean([x[1] for x in metric_values])}

In [5]:
items_df['genres'] = [ast.literal_eval(genres) for genres in items_df['genres']]
items_df['countries'] = [ast.literal_eval(countries) for countries in items_df['countries']]
items_df['staff'] = [ast.literal_eval(staff) for staff in items_df['staff']]

In [6]:
class GradientBoosting(BaseRecommender):
    def __init__(self, iterations: int = 1, learning_rate: float = 0.25, depth: int = 6):
        super().__init__()
        self.mlb = MultiLabelBinarizer()
        self.model = CatBoostRegressor(iterations=iterations, learning_rate=learning_rate, depth=depth, verbose=False)

        
        
        # Бинарные признаки закодируем как 0 и 1, а остальные признаки являются порядковыми, 
        # поэтому можно их просто равномерно нормировать


        self._age_encoder = {
            '18-24': 0.0,
            '25-34': 0.2,
            '35-44': 0.45,
            '45-54': 0.69,
            '55-70': 1.0
        }

        self._income_encoder = {
            'низкий': 0.0,
            'средний': 0.33,
            'высокий': 0.67,
            'очень высокий': 1.0
        }
        
        self._sex_encoder = {
            'Женский': 0.0,
            'Мужской': 1.0
        }
        
        self._education_encoder = {
            'Без образования': 0.0,
            'Среднее': 0.33,
            'Неполное высшее': 0.67,
            'Высшее': 1.0
        }
        
        
    # Закодируем категориальные признаки в соответсвтии со словарями, None заменим средними значениями
    def _prepare_categorical_features(self) -> None:
        self.df['age_category'] = self.df['age_category'].replace(self._age_encoder)
        self.df['age_category'].fillna(self.df['age_category'].mean(), inplace=True)

        self.df['income'] = self.df['income'].replace(self._income_encoder)
        self.df['income'].fillna(self.df['income'].mean(), inplace=True)
        
        self.df['sex'] = self.df['sex'].replace(self._sex_encoder)
        self.df['sex'].fillna(self.df['sex'].mean(), inplace=True)
        
        self.df['education'] = self.df['education'].replace(self._education_encoder)
        self.df['education'].fillna(self.df['education'].mean(), inplace=True)
        
        self.df['kids_flg'].fillna(self.df['kids_flg'].mean(), inplace=True)
        
    def _prepare_movies(self) -> None:
        # Для каждого пользователя оставим только информацию о самых популярных фильмах
        self.df['movie_id'] = [[movie for movie in movies if movie in self.recommendations] for movies in self.df['movie_id']]
        
        # Применим one-hot кодирование
        binarized_column = self.mlb.fit_transform(self.df['movie_id'])
        binarized_column_df = pd.DataFrame(binarized_column, columns=self.mlb.classes_)

        self.df = pd.concat([self.df, pd.DataFrame(binarized_column_df)], axis=1)

        self.df.drop(columns=['movie_id'], inplace=True)
        
        
    def _prepare(self) -> None:
        self._prepare_categorical_features()
        self._prepare_movies()

    def fit(self, train_df: pd.DataFrame, items_df: pd.DataFrame) -> None:
        self.recommendations = set(train_df['movie_id'].value_counts().index.values[:100])
        
        self.df = train_df.groupby("user_id").agg({"movie_id": list}).reset_index()
        self.df = pd.merge(self.df, users_df, on='user_id', how='left')
        
        self.grouped_df = train_df.groupby("user_id").agg({"movie_id": list}).reset_index()
        
        self.matrix = pd.DataFrame(self.mlb.fit_transform(self.df['movie_id']), columns=self.mlb.classes_)
        
        self._prepare()
        self.trained = True
        
    def _predict(self, ind) -> None: # Функция предсказания для пользователей с номерами от ind до ind+test_rows
        current_test_df = self.test_df.iloc[ind: min(ind + self.test_rows, len(self.test_df))]
        
        # Разделим данные на одучающую и тестовую части
        mask = self.df['user_id'].isin(current_test_df['user_id'])
        
        train_indices = self.df.index[~mask]
        test_indices = self.df.index[mask]

        X_train = self.df.iloc[train_indices]
        X_test = self.df.iloc[test_indices]
        
        for movie in self.matrix.columns: # Для каждого фильма обучим модель и сделаем предсказание
            target_matrix = self.matrix[[movie]]
            
            y_train = target_matrix.iloc[train_indices]
            
            # Если ни один пользователь из обучающей части не посмотрел фильм, то обучать модель нет смысла
            if len(y_train.iloc[:, 0].unique()) == 1: 
                self.ratings.loc[ind:min(ind + self.test_rows, len(self.test_df)), y_train.columns] = 0
                continue
                
            # Если фильм популярный, то он присутствует как признак. 
            # В таком случае его нужно удалить, чтобы корректно обучить модель
            if movie in self.recommendations:
                train_pool = Pool(data=X_train.drop(columns=[movie]), label=y_train)
                test_pool = Pool(data=X_test.drop(columns=[movie]))
            else:
                train_pool = Pool(data=X_train, label=y_train)
                test_pool = Pool(data=X_test)                

            self.model.fit(train_pool)

            predictions = self.model.predict(test_pool)
            
            # Запишем предсказания в датафрейм ratings. Чем выше ratings[user_id][movie], 
            # тем вероятнее пользователь user_id посмотрит фильм movie
            self.ratings.loc[ind:min(ind + self.test_rows, len(self.test_df)) - 1, y_train.columns] = predictions.reshape(-1, 1)

    def predict(self, df: pd.DataFrame, test_parts: int = 1) -> list:
        '''
        test_parts в данном методе - это количество частей, на которые мы будем делить тестовую часть. 
        Т.е. если test_parts == len(test_part), то для каждого пользователя из тестовой части будем отдельно обучать 
        модель, используя для обучения всех пользователей кроме текущего. Если test_parts == 1, 
        то модель будем обучать только один раз - на всех пользователях кроме тестовых.
        '''

        assert self.trained

        self.test_df = df
        self.test_rows = len(df) // test_parts
        
        # Создаем матрицу, в которой для каждой пары (user_id, movie) будет записано предсказание.
        self.ratings = pd.DataFrame(columns=self.matrix.columns, index=range(len(self.test_df)))
        
        # Делаем предсказания по частям
        for ind in range (0, len(self.test_df), self.test_rows):
            self._predict(ind)
        
        # Если пользователь user_id посмотрел фильм movie, то считаем, что ratings[user_id][movie] = 0,
        # чтобы не рекомендовать фильм, который пользователь уже посмотрел.
        for user in range(len(self.test_df)):
            user_id = self.test_df.iloc[user]['user_id']
            self.ratings.iloc[user][list(self.grouped_df[self.grouped_df['user_id'] == user_id]['movie_id'])[0]] = 0
            
        # Выбираем для каждого пользователя 10 фильмов с наивысшей оценкой.
        self.ratings = self.ratings.astype(float)
        top_10_movies = self.ratings.apply(lambda row: row.nlargest(10).index.tolist(), axis=1)

        return top_10_movies.tolist()

In [7]:
model = GradientBoosting(iterations = 5, learning_rate = 0.25, depth = 6)
model.fit(train_part, items_df)

In [ ]:
%%time
test_part['gb_recs'] = model.predict(test_part, 1)

In [ ]:
evaluate_recommender(df=test_part, model_preds_col="gb_recs")